## Introduction 

The goal of this lab is to learn how to set up and configure Web, Email, and DNS servers and how to capture and analyze their traffic. The lab has several **milestones** with subsections. Make sure you reach each one before advancing to the next.


## System Setup 

This lab will use a system set up as shown in Figure 1, which is defined in the "docker-compose.yml" file.  Additionally, to make things easier, the required containers' ports are mapped to the Docker Host, as shown in Figure 2.

|<img src="figures/setup1.jpg" width="800" height="800" />|
|:--:| 
| *Figure 1: Lab2 System Setup* |



|<img src="figures/setup2.jpg" width="800" height="800" />|
|:--:| 
| *Figure 2: Lab2 Port Mapping* |



   >_You can change the port number on the Docker Host, as long as you avoid conflicts._


* Create and start the containers using the docker-compose file:

   ```bash
    docker-compose up -d --build
    ```

    >_The first time you run this command, it takes a while to build the images (less than 5 minutes). Use that time to read the "docker-compose.yml" to understand how the system is set up._
    


* You can connect to any container using SSH:

   ```bash
    ssh ttm4200@127.0.0.1 -p [Mapped Port in the Docker Host]
    ```


* To copy files from or to containers, use `scp` or the mounted volumes. Read through the "docker-compose.yml" to find out which directories are mounted.

* Inside the containers, the password for the root and the user "ttm4200" is setup to "ttm4200". If you want, you can change it in the file "ttm4200_base/utilities.sh".

* To save your configuration in any container, run the script: `sudo sh ~/work_dir/save.sh` inside that specific container.

# Milestone 1 -- Web Server 

In this lab, we will use [Nginx](https://www.nginx.com/) (<https://www.nginx.com/>) as a web server.
It is an open-source web server,  which is also used as a reverse proxy, HTTP cache, and load balancer.
As of October 2020, _Nginx_ served 34 percent of all websites, ranking it first above Apache at 27 percent (according to [Netcraft](https://news.netcraft.com/archives/2020/10/21/october-2020-web-server-survey.html) (<https://news.netcraft.com/archives/2020/10/21/october-2020-web-server-survey.html>)).


* _Nginx_ is already pre-installed in the "webserver" image. It is configured to serve a static HTML page. To access this page in your web-browser, run this command in your **VM:**

    ```bash
    sudo sh -c 'echo "127.0.0.1 www.ttm4200.com" >> /etc/hosts'
    ```
    >_This is to bypass DNS resolution in your VM. It basically lets the VM to translate the domain "www.ttm4200.com" to the IP address "127.0.0.1"(localhost) without using the DNS protocol for resolving the domain name._
    
* Access this page by typing in your browser: "www.ttm4200.com:80".

    >_The port number in the domain name isn't necessary since the default HTTP port is 80. However, if you mapped a different HTTP port to your Docker Host, you must use the port after the domain: "www.ttm4200.com:[port]/some/path/"_

 
## 1.1 Creating your own website

* Access the "webserver" container and create a root directory for your website: `sudo mkdir /var/www/teamsite`. This will be the directory from which requests will be served.

* Create a simple HTML page (e.g., containing your team's info) and place it in your website's root directory: `sudo touch /var/www/teamsite/info.html`.

* Create a server block file that tells _Nginx_ how to serve your website's contents: `sudo touch /etc/nginx/sites-available/teamsite`. You can start with this template by filling it with your configuration:

    ```yml
    server {
            # specify the port your webserver will listen to
            listen ====fill in here==== ; 

            # specify root directory of your website
            root ====fill in here==== ;

            # specify the default page (e.g. your HTML page)
            index ====fill in here==== ;

            # Specify your server name. This will be the domain name
            #of your website.
            server_name www.teamsite.com ; 
    }
    ```

   >_By default, Nginx server block configuration files are stored in "/etc/nginx/sites-available" and enabled through symbolic links to  "/etc/nginx/sites-enabled/". You can have multiple configuration files for different websites with different names. The `server_name` directive determines which server block is used to a given request, not the file name._

* Enable your website by creating a symbolic link from your config file to the "sites-enabled" directory, which _Nginx_ reads from during startup.

   ```bash
   sudo ln -s /etc/nginx/sites-available/teamsite /etc/nginx/sites-enabled/teamsite
   ```

* Check the syntax of the _Nginx_ configuration file to make sure that there are no syntax errors `sudo nginx -t`, then restart _Nginx_ (`sudo service nginx restart`).


* To access the web page from a browser, run this command in your **VM:**

    ```bash
    sudo sh -c 'echo "127.0.0.1 www.teamsite.com" >> /etc/hosts'
    ```


* Verify that your website is properly configured using a browser from your host machine ( "www.teamsite.com:80").

In [ ]:
from test_lab2 import TestLab2
check_progress = TestLab2()
check_progress.test_1_1()

## 1.2 Capturing HTTP Traffic

* Access the container "client1" and start two `tmux` panes. In one pane,  start packet capturing with `tcpdump` and save the captured packets to a file named "client1_http.pcap".

* In the second pane, retrieve the HTML page from your website. Run these two commands:
   
   ```bash
   wget http://10.20.30.3 --no-http-keep-alive --header "Host: www.teamsite.com"
   
   wget http://10.20.30.3 --header "Host: www.teamsite.com"
   ```

* Stop the `tcpdump` capturing and copy the capture file to your host machine using `scp`, place the file inside the "lab2" directory. 

In [ ]:
check_progress.test_1_2()

## 1.3 Analyzing HTTP Traffic

* In your VM, open the capture file in Wireshark, and apply the display filter "http" so that only HTTP packets will be displayed.

* Examine the information in one of the HTTP GET requests and its corresponding response. Then answer the following questions:
    * What is the packet number (packet ordinals in the capture file) of the HTTP GET and HTTP response?
    * What is the version of the HTTP client? What version of HTTP is the server running?
    * Is this a persistent or non-persistent connection?
    * How many bytes of content (content-length) are returned in the HTTP response?
    * When was the HTML file that you are retrieving last modified at the server?

In [ ]:
# What is the packet number (packet ordinals in the capture file) of the HTTP GET and HTTP response?
HTTP_GET_packet_number =  #your answer (as an interger)
HTTP_response_packet_number = #your answer (as an interger)

# What is the version of the HTTP client? What version of HTTP is the server running?
client_HTTP_version =  #your answer (as a float)
server_HTTP_version =  #your answer (as a float)

# Is this a persistent or non-persistent connection?
connection_type =  #your answer (as a string of either 'persistent', or 'non-persistent')

# How many bytes of content (content-length) are returned in the HTTP response?
content_length= #your answer (as an interger)


# When was the HTML file that you are retrieving last modified at the server?
last_modified =  ##your answer (as a string of "hours:minutes:seconds")

check_progress.test_1_3(HTTP_GET_packet_number,
                        HTTP_response_packet_number,
                        client_HTTP_version,
                        server_HTTP_version,
                        connection_type,
                        content_length, last_modified)

# Milestone 2 -- Mail Server

The goal of this milestone is to understand and analyze email traffic. For simplicity, the email server is already set up for you.

   >_If you are interested in how to build an email server, you can look at this tutorial: <https://linoxide.com/mail/install-configure-postfix-ubuntu/>. Essentially, we used [Postfix](http://www.postfix.org/) (<http://www.postfix.org/>) as a mail transfer agent (SMTP),  [Dovecot](https://www.dovecot.org/) (<https://www.dovecot.org/>) as an IMAP server, and  [Roundcube](https://en.wikipedia.org/wiki/Roundcube) (<https://en.wikipedia.org/wiki/Roundcube>) as a webmail._


You will use a simple scenario as shown in figure 3, in which you send an email from "Alice@ttm4200.com" to "Bob@ttm4200.com" and receive a reply while capturing all traffic to the mail server. 

|<img src="figures/email.jpg" width="800" height="800"/>|
|:--:| 
| *Figure 3: Email traffic scenario: Alice uses an email client (Thunderbird), thus pushing the message to the mail server using SMTP. Bob retrieves this message using webmail and replies to it. Then Alice retrieves the reply from the mail server using IMAP.* |

* To capture the authentication process, you need to start packet capturing before login. Start a packet capture with `tcpdump` on the "mailserver" and dump it into a "mailserver.pcap" file.

In your __VM__, open Thunderbird an navigate to __Account Settings__$\rightarrow$__Account Actions__$\rightarrow$__Add Mail Account__ and add the email acount of "Alice@ttm4200.com", as shown in figure 4 (password: ttm4200). Then send an email to "Bob@ttm4200.com".

|<img src="figures/alice.png" width="600" height="600"/>|
|:--:| 
| *Figure 4: Thunderbird Setup* |




   >_In this lab, we are using insecure IMAP (over port 143), but it is possible to use IMAPS (IMAP over SSL) in which traffic travels over a secure socket to a secure port (port 993). Therefore, Thunderbird will warn you, as shown in figure 5_


|<img src="figures/security_warning.png" width="400" height="400"/>|
|:--:| 
| *Figure 4: Thunderbird Warning* |

* From a browser, access the webmail page ("www.ttm4200.com/webmail/") and login to "Bob@ttm4200.com (password: ttm4200). Check that it received the email sent by Alice and reply to the message.

* In Thunderbird, check that you received the reply. Then stop packet capturing and copy the capture file to your VM using `scp`, place the file inside the "lab2" directory.



## Analyzing Mail Server Traffic

### SMTP 

* In your VM, open the capture file in Wireshark and apply the "smtp" display filter to show only SMTP-based traffic. 

* Select the packet corresponding to the body of the email sent from Alic to Bob. Then [Follow TCP Stream](https://www.wireshark.org/docs/wsug_html_chunked/ChAdvFollowStreamSection.html) (__Right-click__$\rightarrow$__Follow__$\rightarrow$__TCP Stream__). 

* Examine the SMTP transport process (compare it with the transcript on pages 147-148).

* By closing the dialog, it will apply a display filter that selects all the current stream packets. Then answer the following questions:
    * What are the packet number of SMTP Service Ready Traffic (packet labeled 220) and the client TCP acknowledgment of receiving the Service Ready message?.
    * What is the packet number of SMTP EHLO (extended hello) traffic? This is a packet corresponding to the client initiating a session with the server with an _EHLO_ command.
    * What is the packet number corresponding to transmission of the body of the mail message, where you can read the content of the email in a line-based text SMTP/[IMF] (https://gitlab.com/wireshark/wireshark/-/wikis/IMF)?
    * What is the packet number of SMTP QUIT Traffic? This packet corresponds to a session end, initiated with _QUIT_.
    * What is the packet number of SMTP Closing Traffic (packet labeled 221)?
   

In [ ]:
# What are the packet number of SMTP Service Ready Traffic (packet labeled 220) and the client TCP acknowledgment of receiving the Service Ready message?.
service_ready_packet_number =   # your answer (as an interger)
service_ready_acknowledgement_packet_number =   # your answer (as an interger)

# What is the packet number of SMTP EHLO (extended hello) traffic? 
EHLO_packet_number =   # your answer (as an interger)

# What is the packet number corresponding to transmission of the body of the mail message?
email_body_packet_number =   # your answer (as an interger)

# What is the packet number of SMTP QUIT Traffic? 
QUIT_packet_number =   # your answer (as an interger)

# What is the packet number of SMTP Closing Traffic?
closing_traffic_packet_number=  # your answer (as an interger)

check_progress.test_2_1(service_ready_packet_number, 
                        service_ready_acknowledgement_packet_number, 
                        EHLO_packet_number, 
                        email_body_packet_number, 
                        QUIT_packet_number, 
                        closing_traffic_packet_number)

### IMAP

* Apply a display filter "imap&&imf" to show IMAP and IMF-based traffic. Select the packet corresponding to the body of the IMAP retrieval of the message from Bob (make sure that you see the message in Line-based text). Follow TCP Stream and observe the IMAP connection process: the server sending the capabilities it supports, authentication, login, listing folders, opening a session, fetching messages, idle, ... etc.

* Close the "Follow TCP Stream" window, then answer the following questions:
    * What is the packet number corresponding to the body of the IMAP retrieval of the message from Bob?
    * What is the packet number of the IMAP FETCH command sent by the client before message retrieval from Bob?
    * What is the packet number corresponding to the client requesting login authentication (Request: authenticate PLAIN) and the server response of successful login (Response:  OK)?

In [ ]:
# What is the packet number corresponding to the body of the IMAP retrieval of the message from Bob?
reply_body_packet_number =   # your answer (as an interger)

# What is the packet number of the IMAP FETCH command sent by the client before message retrieval from Bob?
fetch_packet_number=   # your answer (as an interger)

# What is the packet number corresponding to the client requesting login authentication (Request: authenticate PLAIN)? 
auth_request_packet_number =   # your answer (as an interger)

# What is the packet number corresponding to the server response of successful login (Response:  OK).
auth_response_packet_number =   # your answer (as an interger)

check_progress.test_2_2(reply_body_packet_number, fetch_packet_number, auth_request_packet_number, auth_response_packet_number)

### Web-based E-Mail (HTTP)
* Apply the filter "http.request.method == "POST"" to show only HTTP POST-requests. Select the packet corresponding to the message sent from Bob to Alice through webmail.
    * What is the packet number?

In [ ]:
# What is the packet number corresponding to the message sent from Bob to Alice through webmail?
web_email_reply_packet_number =   # your answer (as an interger)
check_progress.test_2_3(web_email_reply_packet_number)

# Milestone 3 -- DNS Server

In this lab, you will set up and run your own authoritative name server.
We will use [BIND9](https://www.isc.org/bind/) (<https://www.isc.org/bind/>), which is an open-source DNS system.

## 3.1 DNS Configuration

###  _BIND9_ Configuration

_BIND9_ is already installed in the "dnsserver" image, but you need to configure it as the __authoritative__ name server for the domain "ttm4200.com".
The configuration files are stored in "/etc/bind/".

* Configure _BIND9_ options (in "/etc/bind/named.conf.options") to:

  * allow queries only from your private network

  * forward unknown queries to other DNS servers (e.g. NTNU's DNS server: "129.241.0.200")

  * enable recursive queries.
  
  * You can start with this template by filling it with your own configurations:

   ```bash
   options {
         # a directory  where BIND stores its cache (previously resolved domain names)
            directory "/var/cache/bind";   

            listen-on port ==== fill in here ==== { ==== fill in here ==== }; 

            allow-query { ==== fill in here ==== };   

            forwarders { ==== fill in here ====  };   

            recursion ==== fill in here ====; 
         };
   ```

### DNS Forward Lookup

The forward lookup [zone](https://en.wikipedia.org/wiki/Zone_file) (<https://en.wikipedia.org/wiki/Zone_file>) stores DNS Record (hostname to IP address relations) for your domain.

* Create a forward lookup for the "ttm4200_net" network, as shown in figure 1. Call the zone file "ttm4200.com.zone" and place it in "/etc/bind/". You can start by filling out (`==== fill in here ====`) this template (note that the semicolon is a comment in zone files):


   ```bash
;$ORIGIN is a standard DNS directive that defines a base name 
;if an $ORIGIN directive is not defined - BIND synthesizes an 
;$ORIGIN from the zone name in the named.conf.local file
$ORIGIN ttm4200.com.  

;Time to Live value for the zone 
$TTL 1h   

;A Start of Authority record that contain administrative information about the zone
;especially regarding zone transfers. 
@               IN  SOA   ns.ttm4200.com. root.ttm4200.com. ( 
                       2019111001; serial
                       1d ; refresh
                       2h ; retry
                       4w ; expire
                       1h ); minimum

;Address record for the webserver ttm4200.com.
@              IN  A    ==== fill in here ====
              IN  AAAA  fd00::3

;Alias records (CNAME) for www.ttm4200.com 
www            IN  CNAME ==== fill in here ====

;Name Server record (NS) for ttm4200.com. that define which servers serve copies of 
;this zone (it must point to an A record and/or AAAA record)
@              IN  NS    ns.ttm4200.com.
ns             IN  A     ==== fill in here ====
              IN AAAA   fd00::2

;Mail Exchange record (MX) for ttm4200.com. that define where email should be sent 
;to and at what priority (it must point to an A record and/or AAAA record)
@              IN  MX    10  mail.ttm4200.com.
mail           IN  A     ==== fill in here ====
              IN  AAAA  fd00::4
   ```

* Add your forward zone to _BIND9_, thus turning it into a Primary Master server for your domain. You can edit the file "/etc/bind/named.conf.local" to do that.

   ```bash
   #     domain name
   zone "ttm4200.com" IN {
            type master;
            # file name of your forward zone (full path)
            file ==== fill in here ====;
   };
   ```

In [ ]:
check_progress.test_3_1()

### 3.2 Capturing DNS Traffic

* Restart _BIND9_ (`sudo service bind9 restart`).



* In the container "client1", configure it to use your DNS server as its nameserver (in "/etc/resolv.conf"):
   ```bash
   nameserver 10.20.30.2
   ```


* In your client container, run the following queries:

  ```bash
  # Get the IP address of `ttm4200.com`
  nslookup -type=A ttm4200.com
  # Determine the authoritative DNS server of `ttm4200.com`
  nslookup -type=SOA ttm4200.com
  # Determine the mail server for `ttm4200.com`
  nslookup -type=MX ttm4200.com
  # Get the IPv6 address of `ttm4200.com`
  nslookup -type=AAAA ttm4200.com
  # Query for any type of record information in the domain `ttm4200.com`
  nslookup -type=any ttm4200.com
  # Performs DNS lookups and displays more details or domain info.
  dig ttm4200.com
  ```

In [ ]:
check_progress.test_3_2()

## 3.3 Analyzing DNS Packets

* Start a packet capturing with `tcpdump` on your DNS server and dump it to a file named "dns.pcap".

* Repeat the previous `nslookup` commands (of subsection 3.2) in 'client1', then stop packet capturing and copy the trace file "dns.pcap" to your host machine and place it in the "lab2" directory.

* Open the capture file with `wireshark`. Apply "dns" display filter to show only DNS-based traffic.

* Examine a DNS query and its corresponding response, then answer the following questions 
    * What packet number (packet ordinals in the capture file) of the DNS query and response?
    * What is the destination port for the DNS query message? What is the source port of the DNS response message?
    * Examine the DNS query message. What "Type" of DNS query is it?

In [ ]:
# What packet number (packet ordinals in the capture file) of the DNS query and response?
DNS_query_packet_number =  #your answer (as an interger)
DNS_response_packet_number =  #your answer (as an interger)

# What is the destination port for the DNS query message? What is the source port of the DNS response message?
DNS_query_destination_port =  #your answer (as an interger)
DNS_response_source_port =  #your answer (as an interger)

# Examine the DNS query message. What "Type" of DNS query is it?
DNS_query_type = # your answer (as a string of: "A", "AAAA", "NS", "SOA", "MX", or "PTR")


check_progress.test_3_3(DNS_query_packet_number, 
                        DNS_response_packet_number, 
                        DNS_query_destination_port, 
                        DNS_response_source_port,
                        DNS_query_type)

## 3.4 DNS Reverse Lookup

Reverse lookup zones are used to resolve IP addresses to host names, rather than host names to IP addresses. For more information on how to create a reverse zone, you can read this [guide](https://www.apnic.net/about-apnic/corporate-documents/documents/resource-guidelines/reverse-zones/) (<https://www.apnic.net/about-apnic/corporate-documents/documents/resource-guidelines/reverse-zones/>).
 
* Create a reverse lookup zone for your private network. Call it "rev-ttm4200.com.zone" and place it in "/etc/bind". You can start by filling out this template:

   ```bash
$ORIGIN 30.20.10.in-addr.arpa.  
$TTL 1h             
; Start of Authority (SOA) record     
@          IN  SOA   ns.ttm4200.com. root.ttm4200.com. ( 
                                   2019111001; serial
                                   1d ; refresh
                                   2h ; retry
                                   4w ; expire
                                   1h ); minimum
;Nameserver (NS) records declaring the nameserver that serve this zone                  
@         IN  NS    ns.ttm4200.com.
;pointer (PTR) record Within the zone, you then need to create domain pointer
;(PTR) records for each IP address.
==== fill in here ====        IN  PTR    ns.ttm4200.com.
==== fill in here ====        IN  PTR    mail.ttm4200.com.
==== fill in here ====        IN  PTR    www.ttm4200.com.

   ```



* Add your reverse zone to _BIND9_. You can fill in the example below and add it to "named.conf.local".

   ```bash
   # reverse domain name
   zone "30.20.10.in-addr.arpa" IN {
            type master;
            # file name of your forward zone (full path)
            file ==== fill in here ====;
   };
   ```

In [ ]:
check_progress.test_3_4()

## 3.5 Analyzing Reverse DNS Packets

* Restart _BIND9_ for the reverse zone to take effect.

* Start a packet capture with `tcpdump` on your DNS server and dump it to a file named "rev_dns.pcap".

* In your "client1" container, perform a reverse DNS lookup of your mail server IP address (e.g. `dig -x 10.20.30.4`).

* Stop the packet capture on the DNS server. Copy the capture file to your host machine, and place it inside the "lab2" directory.

* Open the capture file with `wireshark` and apply a "dns" display filter to show only DNS-based traffic. 

* Examine a DNS query and its corresponding response, then answer the following questions:
    * What packet number (packet ordinals in the capture file) of the DNS query and response?
    * Examine the DNS query message. What "Type" of DNS query is it?


In [ ]:
# What packet number (packet ordinals in the capture file) of the DNS query and response?
rev_DNS_query_packet_number =  #your answer (as an interger)
rev_DNS_response_packet_number =  #your answer (as an interger)

# Examine the DNS query message. What "Type" of DNS query is it?
rev_DNS_query_type = # your answer (as a string of: "A", "AAAA", "NS", "SOA", "MX", or "PTR")


check_progress.test_3_5(rev_DNS_query_packet_number, 
                        rev_DNS_response_packet_number, 
                        rev_DNS_query_type)

## Optional Exercise

* Connect the configured DNS with the Web and Email servers. Capture the traffic from opening a webpage and examine the DNS resolution process.